In [1]:
import os
import os
import itertools
os.environ['OMP_NUM_THREADS'] = '1'
import numpy as np
import mne
from mne_connectivity import symmetric_orth
from hmmlearn import hmm
from scipy.signal import hilbert  # For Hilbert transform
from scipy.signal import resample, butter, lfilter # For downsampling
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import networkx as nx
import seaborn as sns
from scipy.optimize import fminbound

from markovian_helpers_noclass import downsample_with_filtering, apply_orthogonalization, VariationalHMM, fit, elbo, free_energy

optimal_states_arr = []

# defining input and output directory
files_in = '../data/in/subjects/'
files_out = '../data/out/subjects/'


# loading list of subject names from txt file
names = open("./names.txt", "r")
subject_list = names.read().split('\n')
modes = ['EC', 'EO']
for subject in subject_list:
    for mode in modes:
        print(subject, mode)
        #defining input and output directories for each subject and mode
        dir_in = files_in + subject + '/' + mode + '/'
        dir_out = files_out + subject +"/" + mode +'/'

        orthogonalized_data = np.load( dir_out + "orth.npy")


        # Step 2: Determine the Optimal Number of States for the HMM

        # Compute the variance of your features to set a variance floor later
        feature_variances = np.var(orthogonalized_data, axis=0)

        # Choose a small fraction (e.g., 1% or 0.1%) of the maximum variance as the variance floor
        fraction_of_max_variance = 0.05  # Adjust as needed
        variance_floor = fraction_of_max_variance * np.max(feature_variances)

        # Handle NaNs and Infs in features (using masking)
        features = np.mean(orthogonalized_data, axis=2)
        features = np.ma.masked_invalid(features).filled(0)

        # Reshape orthogonalized_data using array.reshape (-1, 1)
        # New shape will be (samples/epochs, labels * sampling frequency)
        reshaped_data = orthogonalized_data.reshape(-1, 1)

        # Reduce dimensionality to speed up HMM fitting
        pca = PCA(n_components=0.99)  # Retain 99% of the variance

        # Fit PCA to the normalized data
        pca_data = pca.fit_transform(reshaped_data)

        # Standardize the PCA-transformed data
        scaler = StandardScaler()
        pca_data = scaler.fit_transform(pca_data)

        # Define the range of hidden states to explore
        state_numbers = range(3, 16)


        # Initialize lists to store models and Free Energies
        elbos = []
        models = []
        free_energies = []

        # Loop through different numbers of hidden states
        for n in state_numbers:
            # Create a Variational HMM model
            print(n)
            
            #initializing fields
            # model = VariationalHMM(n, pca_data)
            n_states = n
            data = pca_data
            # Initialize priors (adjust if you have prior knowledge)
            init_distrib = np.ones(n_states) / n_states
            trans_distrib = np.ones((n_states, n_states)) / n_states
            emission_means = np.random.randn(n_states, data.shape[1])
            emission_covs = np.ones((n_states, data.shape[1]))

            # Fit the model and return q (posterior distribution) after fitting
            q = fit(n_states, data, max_iter=100)

            # Calculate ELBOs (Evidence Lower Bound) and store them: Helps to analyze convergence if needed
            elbo = elbo(q)
            elbos.append(elbo)

            # Calculate and store Free Energy: -ELBO
            free_energy = free_energy(q)  # Assuming 'q' is your final posterior
            free_energies.append(free_energy)
            models.append([n_states, q, elbo, data])

        # Plotting
        plt.bar(state_numbers, free_energies)
        plt.xlabel("Number of States")
        plt.ylabel("Free Energy")
        plt.show()

        # Find the optimal number of states
        optimal_states = state_numbers[np.argmin(free_energies)]
        print(f"Optimal number of states based on Varitional Bayes for Subject {subject}: {optimal_states}")
        optimal_states_arr.append({subject:optimal_states})     



/home/prakhar7/.conda/envs/hmmenv/lib/python3.12/site-packages/numba/core/decorators.py:250: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)
/home/prakhar7/.conda/envs/hmmenv/lib/python3.12/site-packages/numba/core/decorators.py:252: RuntimeWarning: forceobj is set for njit and is ignored
  warnings.warn('forceobj is set for njit and is ignored', RuntimeWarning)


101 EC
3


/home/prakhar7/.conda/envs/hmmenv/lib/python3.12/site-packages/numba/core/dispatcher.py:289: UserWarning: Numba extension module 'numba_scipy' failed to load due to 'ValueError(No function 'betainc' found in __pyx_capi__ of 'scipy.special.cython_special')'.
  entrypoints.init_all()


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1mFailed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mInvalid use of type(CPUDispatcher(<function _emission_logprob at 0x7fff4428c4a0>)) with parameters (array(float64, 1d, C), array(float64, 2d, C), array(float64, 2d, C), int64)
[0m
[0m[1mDuring: resolving callee type: type(CPUDispatcher(<function _emission_logprob at 0x7fff4428c4a0>))[0m
[0m[1mDuring: typing of call at /home/prakhar7/mfprl/sample/neuroconn/markov/markovian_helpers_noclass.py (141)
[0m
[1m
File "markovian_helpers_noclass.py", line 141:[0m
[1mdef e_step(n_states, data, init_distrib, trans_distrib, emission_means, emission_covs, q):
    <source elided>
        for i in range(n_states):
[1m            log_lik = _emission_logprob(data[t], emission_means, emission_covs, i)
[0m            [1m^[0m[0m

[0m[1mDuring: resolving callee type: type(CPUDispatcher(<function e_step at 0x7fff4428c040>))[0m
[0m[1